## Problem 1: Difference fanout

>Given a list of numbers, for each number generate a list of the differences between it and $n_{fanout}$ (known as the **fanout** value) following numbers in the list. Return a list of all the lists generated for each number. For members in last part of the list that have less than $n_{fanout}$ following members, calculate as many differences as possible. For example, suppose we want to compute the difference fanout on the list `[3, 2, 4, 6, 1]` with a fanout value of 3, then we would compute:
 - $3 \rightarrow [2 - 3, 4 - 3, 6 - 3]$
 - $2 \rightarrow [4 - 2, 6 - 2, 1 - 2]$
 - $4 \rightarrow [6 - 4, 1 - 4]$
 - $6 \rightarrow [1 - 6]$
 - $1 \rightarrow []$
 
``` Python
# example behavior
>>> difference_fanout([3, 2, 4, 6, 1], 3)
[[-1, 1, 3], [2, 4, -1], [2, -3], [-5], []]
```

You will want to know about [lists](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html#Lists), [indexing & slicing](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Introducing-Indexing-and-Slicing) lists, and [for-loops](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ForLoops.html) to solve this problem.

For extra credits (and some extra fun!), try to write your function only using [list comprehension](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#List-&-Tuple-Comprehensions). 

### Solution: difference_fanout using for-loops
We will naturally tackle this problem by performing nested for-loops. The outermost for-loop will loop over each number in the list, we will refer to this number as the "base number". We will want the inner for-loop to iterate ahead of the base number so that we can compute the difference of it and its $n_{fanout}$ neighbors. We will need to take care re-initialize our intermediate list of differences for each new base number, otherwise each subtraction will get appended to one long list. 

```python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Number]
            Input list of base numbers.
            
        fanout: int
            Number of neighbors to compute difference against.
        
        Returns
        -------
        List[List[Number]]
    """
    all_fanouts = []  # will store each list of fanouts
    for i, base_number in enumerate(l):
        # `base_fanout` will store the differences between 
        # the base number and successive neighbors
        base_fanout = []  
        for neighbor in l[i+1: i+1+fanout]:
            base_fanout.append(base_number - neighbor)
            
        all_fanouts.append(base_fanout)
    return all_fanouts
```

Note our use of [enumerate](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Iterables.html#Enumerating-iterables); this permits us to simultaneously access our base number, which we use in the subtraction, as well as its index-position within the list `l`, which we use to determine the neighbors. 

Next, you may be concerned that our inner-loop will attempt to iterate beyond the end of the list. Consider the case in which `base_number` is the final element in `l`, thus `l[i+1: i+1+fanout]` would be equivalent to `l[len(l): len(l)+fanout]` - the stopping point for this slice clearly reaches beyond the extent of `l` (assuming `fanout > 0`). Fortunately, this is not an oversight on our part. While indexing a list outside of its bounds will raise an error, recall that [a slice will automatically limit itself to be within bounds of a given sequence](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Handling-out-of-bounds-indices). That is, `l[i+1: i+1+fanout]` actually behaves like `l[max(0, i+1): min(len(l), i+1+fanout)]` (assuming we are dealing only with positive indices). Thus our inner-loop will naturally limit itself to only look ahead as far as it can. In the case that `base_number` is the final element in `l`, the inner-loop will exit immediately, leaving `base_fanout` empty.

This is a somewhat obscure but important point to remember about how slicing works in Python.

### Solution: difference_fanout using list comprehensions
We can make judicious use of nested [list comprehensions](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#List-&-Tuple-Comprehensions) to simplify our solution. Although the syntax may appear to be convoluted at first glance, it permits us proceed without worrying about initializing multiple empty lists and appending to them at the right points in our nested for-loops

``` Python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Union(int,float)]
            Input list
            
        fanout: int
            Number of neighbors to compute difference with
        
        Returns
        -------
        List[List[int]]
    """
    return [[neighbor - base for neighbor in l[i+1:i+1+fanout]] 
            for i,base in enumerate(l)]
```

See that the outermost list comprehension loops over the base number, as did the outer for-loop in the prior solution, and that the innermost list comprehension plays the same roll as the inner for-loop.

There are fewer potential points of failure in this solution, as its conciseness removes the "moving parts" that had to be managed in the previous solution. This should help demonstrate the power of the comprehension expression syntax.  

### Extension
Recall from [earlier](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Functions.html#Functions-are-Objects) that functions are, under the hood, just objects with some special operations that allow you to "call" a function. This means that you can pass other functions as parameters into our function. It is especially powerful since it enables us to generalize the purposes of our functions. For example, we don't have to limit our function to just computing the **difference** between members and their following terms; we can apply **any** *binary operation*. Instead of finding the difference, we can calculate the sum or product or even concatenate two strings for a list of string. The possibility is limitless. 
Armed with this knowledge, we can generalize the code.
```Python
def apply_fanout(l, fanout, op):
    """ Return a list of outputs for each value 
        after applying a binary operation between 
        the value and its following terms
        
        Parameters
        ----------
        l: List[Any]
            Input list
        
        fanout: int
            Number of neighbors to apply the operation with
        
        op: Callable[[Any, Any], Any]
            Any binary operation to be applied to fanout-pairs
            of elements in `l`.
        
        Returns
        -------
        List[List[Any]]
    """
    return [[op(base, neighbor) for neighbor in l[i+1:i+1+fanout]] 
            for i,base in enumerate(l)]
```
Now, we can rewrite `difference_fanout` simply as
``` Python
def difference_fanout(l, fanout):
    def subtract(a, b): return a - b
    return apply_fanout(l, fanout, subtract)
```
We can easily change `subtract` to some other function for a totally different use. 

## Problem 2: within_margin_percentage
> An algorithm is required to test out what percentage of the parts the factory is producing fall within a safety margin of the design specifications. Given a list of values recording the metrics of the manufactured parts, a list of values representing the desired metrics required by the design, and a margin of error allowed by the design, compute what percentage of the values are within the safety margin (<=)

``` Python
# example behavior
>>> within_margin_percentage(desired=[10.0, 5.0, 8.0, 3.0, 2.0],
...                          actual= [10.3, 5.2, 8.4, 3.0, 1.2],
...                          margin=0.5)
0.8
```

See that $4/5$ of the values fall within the margin of error - $1.2$ deviates from $2$ by more than $0.5$. 

Complete the following function; consider the edge case where `desired` and `actual` are empty lists.

```python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[float]
            The desired metrics
        
        actual: List[float]
            The corresponding actual metrics. 
            Assume `len(actual) == len(desired)`
        
        margin: float
            The allowed margin of error
        
        Returns
        -------
        float
            The fraction of values where |actual - desired| <= margin
    """
    # YOUR CODE HERE
    pass
```
### Solution: within_margin_percentage
This problem can solved by simply looping over the pairs actual and desired values and tallying the pairs that fall within the margin:
``` Python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[float]
            The desired metrics
        
        actual: List[float]
            The actual metrics
        
        margin: float
            The allowed margin of error
        
        Returns
        -------
        float
            The fraction of values where |actual - desired| <= margin
    """
    count = 0  # tally of how values are within margin
    total = len(desired)
    for i in range(total):
        if abs(desired[i] - actual[i]) <= margin
            count += 1  # Equivalent to `count = count + 1`
    return count / total if total > 0 else 1.0
```

See that we handle the edge case where `desired` and `actual` are empty lists: the [inline if-else statement](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ConditionalStatements.html#Inline-if-else-statements) `count / total if total > 0 else 1` will return `1` when `total` is 0: 
```python
>>> within_margin_percentage([], [], margin=0.5)
1.0
```
which is arguably the appropriate behavior for this scenario (no values fall outside of the margin). Had we not anticipated this edge case, `within_margin_percentage([], [], margin=0.5)` would raise `ZeroDivisionError`.

It is also possible to write this solution using the built-in `sum` function and a [generator comprehension](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#Creating-your-own-generator:-generator-comprehensions) that filters out those pairs of items that fall outside of the desired margin:

```python
def within_margin_percentage(desired, actual, margin):
    total = len(desired)
    count = sum(1 for i in range(total) if abs(actual[i] - desired[i]) <= margin)
    return  count / total if total > 0 else 1.0
```

It is debatable whether this refactored solution is superior to the original one - it depends largely how comfortable you, and anyone else who will be reading your code, is with the generator comprehension syntax.

## Problem 3: is_palindrome
> A palindrome is a string that reads the same from left to right and from right to left. Strings like `racecar` and `Live on time, emit no evil` are palindromes. Notice how only valid alphanumeric characters are accounted for and how palindromes are not case-sensitive. Given a string, return whether or not it is a palindrome. 

``` Python
# example behavior
>>> is_palindrome("Step on no pets!")
True
>>> is_palindrome("'Tis not a palindrome")
False
```

**Tips**: `str.isalnum()` returns whether or not a string has purely alphanumeric characters (it works for one-character strings too)
``` Python
>>> "I love Python".isalnum()
False
>>> "IlovePython".isalnum()
True
```

### Solution: is_palindrome
``` Python
def is_palindrome(s):
    """ Given a string, determine if it is a palindrome.
        Whitespaces and cases are all ignored. Only
        alphanumeric characters are taken into consideration
        
        Parameters
        ----------
        s: Str
            Input string
        
        Returns
        -------
        bool
    """
    filtered_str = "".join([c.lower() if c.isalnum() else "" for c in s])
    
    # the center element in an odd number of characters does not need
    # to be considered
    for i in range(len(filtered_str)//2):
        if filtered_str[i] != filtered_str[-(i+1)]:
            return False
    
    # passing the for loop means all the pairs match up
    return True
```

## Problem 4: funny_text
> You're trying to manipulate a block of text for some fun effects. Tally up the number of occurrences of each word in the block of text; then replace the least common word with the most common word, the second least common with the second most common, and so on. Lowercase everything, but be sure to retain any non-alphabetical characters. Two words are two continuous blocks of alphabetical letters separated by some non-alphabetical character in between.

``` Python
# example behavior
>>> funny_text('I’m selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can’t handle me at my worst, then you sure as hell don’t deserve me at my best.')
'best’can if, but don to hard times. best control of, best out am mistakes make don deserve insecure little a hell. impatient selfish as m’sure hell then deserve worst my, me as t you handle and’sure at then deserve worst i.'
```

**Tips**: look into `Collections.Counter` for tallying words. Read up on its documentation for useful methods. Also, 

### Solution: funny_text
``` Python
from collections import Counter
import re
def funny_text(s):
    """ Takes a string and tally number of occurrecnes each unique 
        word. Replace the most common word with the least, and the
        second most with the second least, and so on. String is
        lowercased, and all other characters are retained
        
        Parameters
        ----------
        s: Str
            The input text
        
        Returns
        -------
        Str
    """
    s = s.lower()
    # [^a-z]+ is a regular expression pattern that looks for any
    # one or multiple continuous characters that are not from a-z. 
    # We use it as a delimiter to split the string up into words
    cleaned = re.split("[^a-z]+", s)
    
    # clean up any leftover empty strings
    cleaned = [w for w in cleaned if w != ""]
    c = Counter(cleaned)
    
    # find mapping from most common to least common, etc
    most_commons = list(c.most_common())
    mapping = dict() 
    for i,v in enumerate(most_commons):
        mapping[v[0]] = most_commons[len(most_commons)-(i+1)][0]
    
    # replace words
    new = "" # new string to build up from
    current_i = 0 # keep track of where we are on s
    
    for w in cleaned:
        i = s.find(w, current_i)
        # if not found, quit
        if i == -1:
            break
        new += s[current_i:i] # add up the non-alphabetical chars
        new += mapping[w] # add the new word
        current_i = i+len(w) # index points to end of current word
    
    if current_i < len(s): # if s ends with non-alphabetical
        new += s[current_i:] # add the rest
    
    return new
```

There are a number of hacks and tricks used in this solution. `re.split` splits up a string into a list of strings that are separated by special delimiters in the original string. In this case, `re.split("[^a-z]+", s)` identifies any block of characters not within a-z and make a cut there. The result is a list of words, without these non-alphabetical characters. 

The construction of the output string is also tricky. We already created a mapping from old words to new words, and we now just have to apply the mapping to every word. `current_i` keeps track of the index we're at in the original string. Each iteration of the for loop we look for the next word, concatenating any non-alphabetical characters in between two words and adding the new word's corresponding replacement. 

## Problem 5: concat_to_str
> Sometimes it is very important to handle different object types of inputs differently in a function. Let's say you're appending a list of objects into one single string. If the object is an integer, convert it into a string by spelling out each digit in base-10 in this format:
`142` $\rightarrow$ `one-four-two`; `-12` $\rightarrow$ `neg-one-two`. If the object is a float, just append it's integer part (obtained by rounding down) the same way and the string `"and float"`:
`12.324` $\rightarrow$ `one-two and float`. If the object is a string, keep it as is. If the object is `None`, switch to a new line. If the object is any other type, return `None`. Each object's transcription in **one line** is separated by `" | "`, and the result should be one large string. 

``` Python
# example behavior
>>> s = concat_to_str([12,-14.23,"Some magic number", None,
>>>                    "Aha", 10.1, 1, 2, None, 5])
>>> print(s)
one-two | neg-one-four and float | Some magic number
Aha | one-zero and float | one | two
five
```

**Tips**: checkout the `isinstance` function introduced [here](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html) for different type handling. Also, consider creating a helper function for the conversion from integer to our special-format string, since we have to do it twice. It's always good to extrapolate repeated tasks into functions. You'll also need to hard-code the conversion from each digit to its English spellout. 

### Solution: concat_to_str
``` Python
def _int_to_str(n):
    """ Takes an integer and formats it into a special string 
        e.g. 142 -> "one-four-two"
             -12 -> "neg-one-two"
    """
    mapping = {"0":"zero","1":"one","2":"two","3":"three",
               "4":"four","5":"five","6":"six","7":"seven",
               "8":"eight","9":"nine","-":"neg"}
    
    out = []
    for digit in list(str(n)):
        out.append(mapping[digit])
    
    return "-".join(out)
    
def concat_to_str(l):
    """ Takes a list of objects and special-handles each 
        to concatenate into a long string. 
        Object can be a str, int, float, or None
        
        Parameters
        ----------
        l: List[Union(str, int, float, None)]
            Input list
        
        Returns
        -------
        Union(str, None)
    """
    # output string
    out = ""
    
    # current line's list; will reset after 
    # reaching a Newline (None)
    current = []
    for item in l:
        if isinstance(item, int):
            current.append(_int_to_str(item))
        elif isinstance(item, float):
            # int(any_float) rounds the float down 
            # to get an integer
            current.append(_int_to_str(int(item)) + " and float")
        elif isinstance(item, str):
            current.append(item)
        elif item is None:
            # create a new line and append the current 
            # line's content
            out += " | ".join(current) + "\n"
            # reset
            current = []
        else:
            return None
    
    # no need to check if the last item in `l` is None
    # since if so `current` would be empty and nothing
    # would be appended
    out += " | ".join(current)
    
    return out
```